# 중간고사 5번: 유방암 종양 예측 문제
*Binary classification*   

- CUDA를 사용하도록 했다.
- binary-cross-entropy를 내장 함수로 바꿨다.
- 학습시에 momentum을 `0.8`로 적용하였다.
- Normalize에 sklearn 함수를 썼다.

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 2020-ai-exam-cancer-revisit

In [0]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler # For Normalization

In [0]:
torch.manual_seed(1)
device = torch.device('cuda')

# Load Data

In [20]:
xy_train = pd.read_csv('2020.AI.cancer-train.csv')
xy_train.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,0,8.734,16.84,55.27,234.3,0.10390,0.07428,0.00000,0.00000,0.1985,0.07098,0.5169,2.079,3.167,28.85,0.015820,0.019660,0.00000,0.000000,0.01865,0.006736,10.17,22.80,64.01,317.0,0.1460,0.1310,0.0000,0.00000,0.2445,0.08865,NaN
1,0,11.360,17.57,72.49,399.8,0.08858,0.05313,0.02783,0.02100,0.1601,0.05913,0.1916,1.555,1.359,13.66,0.005391,0.009947,0.01163,0.005872,0.01341,0.001659,13.05,36.32,85.07,521.3,0.1453,0.1622,0.1811,0.08698,0.2973,0.07745,NaN
2,1,14.950,17.57,96.85,678.1,0.11670,0.13050,0.15390,0.08624,0.1957,0.06216,1.2960,1.452,8.419,101.90,0.010000,0.034800,0.06577,0.028010,0.05168,0.002887,18.55,21.43,121.40,971.4,0.1411,0.2164,0.3355,0.16670,0.3414,0.07147,NaN
3,1,17.350,23.06,111.00,933.1,0.08662,0.06290,0.02891,0.02837,0.1564,0.05307,0.4007,1.317,2.577,44.41,0.005726,0.011060,0.01246,0.007671,0.01411,0.001578,19.85,31.47,128.20,1218.0,0.1240,0.1486,0.1211,0.08235,0.2452,0.06515,NaN
4,0,11.060,17.12,71.25,366.5,0.11940,0.10710,0.04063,0.04268,0.1954,0.07976,0.1779,1.030,1.318,12.30,0.012620,0.023480,0.01800,0.012850,0.02220,0.008313,11.69,20.74,76.08,411.1,0.1662,0.2031,0.1256,0.09514,0.2780,0.11680,NaN


In [35]:
x_train = xy_train.iloc[:, 1:-1]
y_train = xy_train.iloc[:, :1]

print(x_train.shape)
print(y_train.shape)

(398, 30)
(398, 1)


In [0]:
y_train.head()

In [37]:
x_train = np.array(x_train)
y_train = np.array(y_train)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)   # X Normalize

pd.DataFrame(x_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,0.340696,0.377826,0.335398,0.219257,0.477279,0.291350,0.209263,0.244954,0.383043,0.269176,0.107131,0.244861,0.100982,0.064343,0.205243,0.173291,0.102867,0.284249,0.232353,0.165833,0.333528,0.357754,0.321342,0.230104,0.367543,0.219128,0.222727,0.397540,0.266706,0.192630
std,0.167836,0.184034,0.168693,0.150485,0.155025,0.182354,0.187014,0.196334,0.135719,0.148172,0.101203,0.157625,0.097176,0.084435,0.142627,0.133211,0.087205,0.145175,0.147679,0.129111,0.191868,0.168294,0.188846,0.184946,0.165391,0.162435,0.174519,0.229624,0.121017,0.125878
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.226655,0.240717,0.220890,0.120749,0.367248,0.151925,0.066858,0.100373,0.288889,0.165175,0.042178,0.132551,0.040074,0.020250,0.118144,0.078525,0.047860,0.186418,0.136679,0.079512,0.205258,0.231444,0.191179,0.110043,0.255041,0.110415,0.089756,0.220782,0.190371,0.110111
50%,0.301907,0.358088,0.297837,0.171025,0.470457,0.249469,0.144201,0.165905,0.372980,0.239364,0.074108,0.214605,0.072492,0.032481,0.174239,0.134535,0.085681,0.263936,0.200231,0.131200,0.276984,0.350877,0.263905,0.162047,0.354793,0.173130,0.184185,0.339656,0.248965,0.163813
75%,0.424961,0.488903,0.417801,0.277287,0.573151,0.383518,0.315077,0.372875,0.452904,0.343987,0.134021,0.315219,0.124099,0.073537,0.255771,0.227232,0.140059,0.358252,0.280208,0.215721,0.441171,0.475304,0.428420,0.307678,0.462681,0.300172,0.310423,0.570704,0.320619,0.243900
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

# Train Model

In [105]:
W = torch.zeros((30, 1), requires_grad=True, device='cuda')
b = torch.zeros(1, requires_grad=True, device='cuda')
x_train = x_train.cuda()
y_train = y_train.cuda()

Epochs = 200000
lr = 1e-1

sig = torch.nn.Sigmoid()
optimizer = optim.SGD((W, b), lr=lr, momentum=0.8) #Implement Momentum

for epoch in range(1, Epochs+1):
    h = sig(x_train.mm(W)+b)
    cost = F.binary_cross_entropy(h, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch == 1 or epoch%5000 == 0:
        print('Epoch: {:4d}/{} Cost: {:.6f}'.format(epoch, Epochs, cost))

Epoch:    1/200000 Cost: 0.693147
Epoch: 5000/200000 Cost: 0.054148
Epoch: 10000/200000 Cost: 0.042994
Epoch: 15000/200000 Cost: 0.037530
Epoch: 20000/200000 Cost: 0.034060
Epoch: 25000/200000 Cost: 0.031601
Epoch: 30000/200000 Cost: 0.029744
Epoch: 35000/200000 Cost: 0.028276
Epoch: 40000/200000 Cost: 0.027079
Epoch: 45000/200000 Cost: 0.026077
Epoch: 50000/200000 Cost: 0.025222
Epoch: 55000/200000 Cost: 0.024479
Epoch: 60000/200000 Cost: 0.023825
Epoch: 65000/200000 Cost: 0.023243
Epoch: 70000/200000 Cost: 0.022719
Epoch: 75000/200000 Cost: 0.022243
Epoch: 80000/200000 Cost: 0.021807
Epoch: 85000/200000 Cost: 0.021406
Epoch: 90000/200000 Cost: 0.021034
Epoch: 95000/200000 Cost: 0.020688
Epoch: 100000/200000 Cost: 0.020363
Epoch: 105000/200000 Cost: 0.020058
Epoch: 110000/200000 Cost: 0.019771
Epoch: 115000/200000 Cost: 0.019498
Epoch: 120000/200000 Cost: 0.019239
Epoch: 125000/200000 Cost: 0.018993
Epoch: 130000/200000 Cost: 0.018758
Epoch: 135000/200000 Cost: 0.018533
Epoch: 140000/

# Test

In [0]:
x_test = pd.read_csv('2020.AI.cancer-test.csv')

In [0]:
x_test = x_test.iloc[:, :30]
x_test = np.array(x_test)
x_test = scaler.transform(x_test)   # Not fit_transform!!!
x_test = torch.FloatTensor(x_test).cuda()

In [108]:
pred = sig(x_test.mm(W)+b)
pred[:5]

tensor([[1.7497e-03],
        [3.7978e-10],
        [3.4520e-05],
        [5.3386e-06],
        [8.1357e-07]], device='cuda:0', grad_fn=<SliceBackward>)

In [0]:
pred = pred.detach().cpu().numpy()
pred[pred>0.5]=1.0
pred[pred<=0.5]=0.0

In [0]:
pred=pred.astype(np.uint32)
id=np.array([i for i in range(pred.shape[0])]).reshape(-1, 1).astype(np.uint32)
result=np.hstack([id, pred])

submit = pd.DataFrame(result, columns=['id', 'diagnosis'])
submit.to_csv('submit.csv', index=False)

In [73]:
! kaggle competitions submit -c 2020-ai-exam-cancer-revisit -f submit.csv -m 'First submit'

100% 929/929 [00:02<00:00, 348B/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 64, in main
    print(out, end='')
UnicodeEncodeError: 'latin-1' codec can't encode characters in position 34-37: ordinal not in range(256)
